## О данном ноутбуке
### Этот ноутбук к проекту Ford vs Ferrari.

Цели проекта:

На основе baseline и данных в нём рекомендаций повысить точность модели.

In [1]:
# проверяем, что видеокарта подключена
!nvidia-smi -L

In [2]:
# подготавливаем данные
!mkdir /kaggle/temp #папка для временных файлов
!unzip -q -o /kaggle/input/sf-dl-car-classification/train.zip -d /kaggle/temp
!unzip -q -o /kaggle/input/sf-dl-car-classification/test.zip -d /kaggle/temp

In [3]:
# выгрузим версии установленных библиотек
!pip freeze > 'requirements.txt'

In [4]:
# импорты
import os, re, math, random, time, gc, string, pickle, shutil, pathlib, itertools, sys
import numpy as np, pandas as pd, matplotlib.pyplot as plt, PIL
import tensorflow as tf
#import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras import *
from tensorflow.keras.activations import *
from tensorflow.keras.applications import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.layers import *
from tensorflow.keras.layers.experimental.preprocessing import *
from tensorflow.keras.losses import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.optimizers.schedules import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from IPython.display import clear_output
from tqdm.notebook import tqdm
from keras.models import load_model

print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)
print('Keras        :', tf.keras.__version__)

## Основные настройки

In [5]:
# for model
EPOCHS               = 12 #best 12
BATCH_SIZE           = 12 #best 12
VAL_SPLIT            = 0.05 #best 0.05
IMG_SIZE             = 224 #best 520
IMG_CHANNELS         = 3
INPUT_SHAPE          = (IMG_SIZE, IMG_SIZE, IMG_CHANNELS)

# for augmentation
WIDTH_SHIFT_RANGE    = 0.1
HEIGHT_SHIFT_RANGE   = 0.1
HORIZONTAL_FLIP      = True
VERTICAL_FLIP        = False
ROTATION_RANGE       = 10
BRIGHTNES_RANGE      = (0.5, 1.5)
RESCALE              = 1
SHEAR_RANGE          = 0.2
ZOOM_RANGE           = 0.1

# path to files
PATH = '/kaggle/'
INPUT_PATH = PATH+'input/sf-dl-car-classification/'
DATA_PATH = PATH+'temp/'
TRAIN_PATH = DATA_PATH+'train/'
SUB_PATH = DATA_PATH+'test_upload/'

# seed
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
os.environ['PYTHONHASHSEED'] = '0'
# rn.seed(RANDOM_SEED)

## EDA / Анализ данных

In [6]:
sample_submission = pd.read_csv(INPUT_PATH+'sample-submission.csv')
sample_submission.head()

In [7]:
sample_submission.info()

In [8]:
train_df = pd.read_csv(INPUT_PATH+'train.csv')
train_df.head()

In [9]:
train_df.info()

In [10]:
train_df.Category.value_counts()

In [11]:
train_df.Category.hist()

Распределение классов достаточно равномерное — это хорошо.

In [12]:
plt.figure(figsize=(12,8))
random_image = train_df.sample(n=9)
random_image_paths = random_image['Id'].values
random_image_cat = random_image['Category'].values
for index, path in enumerate(random_image_paths):
    im = PIL.Image.open(TRAIN_PATH+f'{random_image_cat[index]}/{path}')
    plt.subplot(3,3, index+1)
    plt.imshow(im)
    plt.title('Class: '+str(random_image_cat[index]))
    plt.axis('off')
plt.show()

Посмотрим на примеры картинок и их размеры, чтобы понимать, как их лучше обрабатывать и сжимать.

In [13]:
image = PIL.Image.open(TRAIN_PATH+'/5/211385.jpg')
imgplot = plt.imshow(image)
plt.show()
image.size

Выводы:

* Для классификации представлены фотографии 10 категорий авто
* Категории сбалансированны по кол-ву изображений
* Размеры фотографий различаются, но в основном 640 на 480
* Всего 22236 фото в том числе:15561 фото в трейне и 6675 в тесте

Кол-во фото в трейне может не хватить для хорошего обучения модели нейронной сети, поэтому будем применять различные виды аугментации данных

## Подготовка данных

Наверняка вы помните, что аугментация данных важна, когда мы работаем с небольшим датасетом. Это как раз наш случай.

In [14]:
train_datagen = ImageDataGenerator(
#     rescale=1. / 255,
    validation_split=VAL_SPLIT,
    width_shift_range = WIDTH_SHIFT_RANGE,
    height_shift_range = HEIGHT_SHIFT_RANGE,
    horizontal_flip=HORIZONTAL_FLIP,
    rotation_range=ROTATION_RANGE,
    shear_range=SHEAR_RANGE,
    brightness_range=BRIGHTNES_RANGE,
    zoom_range=ZOOM_RANGE,
    vertical_flip=VERTICAL_FLIP,
)

val_datagen = ImageDataGenerator(
#     rescale=1. / 255,
    validation_split=VAL_SPLIT,
)

sub_datagen = ImageDataGenerator(
#     rescale=1. / 255,
    width_shift_range=WIDTH_SHIFT_RANGE, 
    height_shift_range=HEIGHT_SHIFT_RANGE,
    horizontal_flip=HORIZONTAL_FLIP
)

### Генерация данных

In [15]:
# Завернем наши данные в генератор:

train_generator = train_datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=RANDOM_SEED,
    subset='training'
)

val_generator = val_datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=RANDOM_SEED,
    subset='validation'
)

sub_generator = sub_datagen.flow_from_dataframe( 
    dataframe=sample_submission,
    directory=SUB_PATH,
    x_col="Id",
    y_col=None,
    shuffle=False,
    class_mode=None,
    seed=RANDOM_SEED,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE
)

In [16]:
def show_first_images(generator, count=6, labels=True, figsize=(20, 5), normalized=False):
    generator = itertools.islice(generator, count)
    fig, axes = plt.subplots(nrows=1, ncols=count, figsize=figsize)
    for batch, ax in zip(generator, axes):
        if labels:
            img_batch, labels_batch = batch
            img, label = img_batch[0], np.argmax(labels_batch[0]) #берем по одному изображению из каждого батча
        else:
            img_batch = batch
            img = img_batch[0]
        if not normalized:
#             img = img.astype(np.float)
            img = img.astype(np.uint8)
        ax.imshow(img)
        # метод imshow принимает одно из двух:
        # - изображение в формате uint8, яркость от 0 до 255
        # - изображение в формате float, яркость от 0 до 1
        if labels:
            ax.set_title(f'Class: {label}')
    plt.show()

    
print('Train:')
show_first_images(train_generator)
print('Val:')
show_first_images(val_generator)
print('Sub:')
show_first_images(sub_generator, labels=False)

## Построение модели

Загружаем предобученную сеть EfficientNetB3:

In [17]:
# base_model = Xception(weights='imagenet', include_top=False, input_shape=INPUT_SHAPE) #95.37%
# base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=INPUT_SHAPE) #95.75%
# base_model = NASNetLarge(weights='imagenet', include_top=False, input_shape=INPUT_SHAPE) #10.00% IMG_SIZE = 331
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=INPUT_SHAPE) #96.14% #97.54 with big picture
# base_model.summary()

In [22]:
# base_model.summary()
# # Рекомендация: Попробуйте и другие архитектуры сетей

In [20]:
# Устанавливаем новую "голову" (head)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dropout(0.25)(x)
x = Dense(256, activation = 'relu')(x)
x = BatchNormalization()(x)
predictions = Dense(10, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(learning_rate=ExponentialDecay(0.0009, decay_steps=100, decay_rate=0.9)),
              metrics='accuracy')

# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# x = Dense(256, activation='relu')(x)
# x = Dropout(0.25)(x)
# x = BatchNormalization()(x)
# predictions = Dense(10, activation='softmax')(x)
# model = Model(inputs=base_model.input, outputs=predictions)
# model.compile(loss='categorical_crossentropy',
#               optimizer=optimizers.Adam(learning_rate=ExponentialDecay(0.001, decay_steps=100, decay_rate=0.9)),
#               metrics='accuracy')

# model.summary()

In [23]:
# model.summary()

## Обучение модели

Добавим ModelCheckpoint чтоб сохранять прогресс обучения модели и можно было потом подгрузить и дообучить модель.

In [24]:
mcheckpoint = ModelCheckpoint('best_model.hdf5', monitor='val_accuracy', verbose=1, mode='max', save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
es = EarlyStopping(monitor='val_loss', patience=5)
callbacks_list = [mcheckpoint, es]

In [25]:
history = model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        validation_data=val_generator, 
        validation_steps=len(val_generator),
        epochs=EPOCHS,
        callbacks=callbacks_list
)

In [26]:
# сохраним итоговую сеть и подгрузим лучшую итерацию в обучении (best_model)
model.save('last_model.hdf5')
model.load_weights('best_model.hdf5')

In [27]:
scores = model.evaluate(val_generator, steps=len(val_generator), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))
os.rename('best_model.hdf5', f'best_model_{round(scores[1]*100, 2)}.hdf5')

Посмотрим на графики обучения:

In [28]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

## Предсказание на тестовых данных

In [29]:
predictions = model.predict(sub_generator, verbose=1).argmax(axis=1)
submission = pd.DataFrame({
    'Id': sub_generator.filenames,
    'Category': predictions
}, columns=['Id', 'Category'])
submission.to_csv('submission.csv', index=False)

## Test Time Augmentation (TTA)

In [30]:
sub_generator.reset()
predictions_tta = []
for _ in range(EPOCHS):
    predictions_tta.append(model.predict(sub_generator, verbose=1))
    sub_generator.reset()
predictions_tta = np.mean(np.array(predictions_tta), axis=0).argmax(axis=1)
submission_tta = pd.DataFrame({
    'Id': sub_generator.filenames,
    'Category': predictions_tta
}, columns=['Id', 'Category'])
submission_tta.to_csv('submission_tta.csv', index=False)

## Ансамблирование

In [31]:
# Возьмем ансабль из лучшей и последней модели. Больший вес отдадим лучшей модели, т.к. у нее лучшая метрика
model_2 = load_model('last_model.hdf5')
sub_generator.reset()
predictions_tta_ansemble = []
for _ in range(EPOCHS):
    predictions_tta_ansemble.append(0.6*model.predict(sub_generator, verbose=1) \
                                    + 0.2*model_2.predict(sub_generator, verbose=1))
    sub_generator.reset()
predictions_tta_ansemble = np.mean(np.array(predictions_tta_ansemble), axis=0).argmax(axis=1)
submission_tta_ansemble = pd.DataFrame({
    'Id': sub_generator.filenames,
    'Category': predictions_tta_ansemble
}, columns=['Id', 'Category'])
submission_tta_ansemble.to_csv('submission_tta_ansemble.csv', index=False)

## Заключение

* Модели показывают хороший результат при адекватном количестве эпох и более высоком разрешении фотографий
* Для повышения качества нейронной сети был пременён метод transfer-learning
* Хороший результат показала сеть EfficientNet B3
* Также для повышения качества сети была применена аугментация данных с помощью ImageDataGenerator
* Для улучшения предсказаний на сабмите применён Test-time augmentations (TTA).

Для улучшения модели возможны следующие пути решения:
* Применение transfer learning с fine-tuning
* Настройка LR, optimizer, loss
* Подбор других переменных (размер картинки, батч и т.д.)
* Использование иных архитектур сетей (а не только Xception) или их ансамбли. Примеры SOTA на ImageNet
* Использование Batch Normalization и настройка архитектуры “головы”
* Применение других функций callback Keras https://keras.io/callbacks/
* Использование разных техник управления Learning Rate (https://towardsdatascience.com/finding-good-learning-rate-and-the-one-cycle-policy-7159fe1db5d6 (eng) http://teleported.in/posts/cyclic-learning-rate/ (eng))
* Использование более продвинутых библиотек аугментации изображений (например, Albumentations)